In [ ]:
import json
import os

import src.data.Dataset as dt

file_path = os.path.abspath(
    "resources/output_json/2024-05-21_21-49_0-1/2024-05-21_21-49_0-1.json"
)
csv_file = os.path.abspath("data/processed/10000Train.csv")
dataset_files = os.path.abspath("resources/input/emptyNonEmptyDataset")

with open(file_path, "r") as file:
    data = json.load(file)

dataset = dt.load_from_csv(csv_file)
dataset = dt.convert_csv_to_abstract(dataset, dataset_files)

file_name_list = dataset["file_name"].tolist()

detection_label = []
false_positive = []
for image in data["images"]:
    file = image["file"]
    is_file_in_dataset = file in dataset["file_name"].values
    if is_file_in_dataset:
        pos = file_name_list.index(file)
        label = 0
        for detection in image["detections"]:
            if detection["category"] != "0":
                label = 1
                break
        detection_label.append(label)
        false_positive.append(int(dataset["label"].iloc[pos] != label))
dataset["detection_label"] = detection_label
dataset["is_false_positive"] = false_positive
total_false_positive = dataset["is_false_positive"].sum()

path = folder_name + os.sep + name + ".csv"
dt.dataset_to_csv(dataset, path)